## Dummy Q-Learning의 문제점 : 내가 낸 답이 최선의 선택이 아닐 수 있다.

#### 이를 해결하기위해 안가본 곳도 가봐야한다.(더 안좋은 선택일수도 있지만, 도전한다.)

<img src="../img/Exploit&Exploration.png" alt="Exploration" width="500">

## Exploit and Exploration 방법

### 1. E-greedy example
```python
e = 0.1
if rand < e:
    a(action) = random
else:
    a = argmax(Q(s,a))
```

### 2. Decaying E-greedy example
: 뒤로 갈수록 영향력이 줄어든다.
```python
for i in range(num_episodes):
    e = 0.1 / ((i / 100) + 1)
    while not done:
        # choose an action by a greedy
        if np.random.rand(1) < e:
            action = env.action_space.sample() # 상하좌우 random으로 갑니다
        else:
            action = np.argmax(Q[state, :]) # 가장 값이 높은 곳으로 갑니다.
```

### 3. Add Random Noise
: random value를 더해준다. <br>
`a = argmax(Q(s,a) + random_values)` or `a = argmax(Q(s,a) + random_values / (i+1)) : decaying 적용`

##### example
: **그냥 했다면 2번째로 진행했겠지만, random noise를 더해줘서 3번째 선택하는 결과가 나옴.** <br>
a = argmax([0.5, 0.6, 0.3, 0.2, 0.5] + [0.1, 0.2, 0.7, 0.3, 0.1]) <br>
a = argmax([0.6, 0.8, 1.0, 0.5, 0.6])

```python
action = np.argmax(Q[state, :] + np.random.randn(1, env.action_space.n) / (i + 1))
```

---
### 과연 아래의 경우 1번 경로로 가야하나, 2번 경로로 가야하나?
<img src="../img/Discounted_Reward.PNG" alt="discounted_reward" width="500">

### 이 경우 discounted reward를 적용
: 나중에 받게 되는 reward를 깎는다. <br>
$\hat{Q}(s,a) = r + \gamma * \underset{a^`}{\operatorname{max}} \hat{Q}(s^`,a^`)$
##### example
$\hat{Q}(s,a) = r + 0.9 * \underset{a^`}{\operatorname{max}} \hat{Q}(s^`,a^`)$

#### 다른 형태로 나타내보면....
$R_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} +... \gamma^{n-t} r_n$ <br>
$R_t = r_t + \gamma R_{t+1}$

```python
# discount factor
dis = 0.99
Q[state,action] = reward + dis * np.max(Q[new_state, :])
```

### 전체 코드

```python
import gym
import gym.envs.registration import register

register(
    id = 'FrozenLake-v3',
    entry_point = 'gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name': '4x4',
            'is_slippery': False}
)
env = gym.make('FrozenLake-v3')

# initialize table with all zeros
Q = np.zeros([env.observation_space.n, env.action_space.n])
# discount factor
dis = 0.99
num_episodes = 2000

# create lists to contain total rewards and steps per episode
rLIst = []

for i in range(num_episodes):
    # reset environment and get first new observation
    state = env.reset()
    rAll = 0
    done = False
    
    # The Q-Table learning algorithm
    while not done:
        # choose an action by greedily with noise picking from Q table
        action = np.argmax(Q[state, :] + np.random.randn(1,env.action_space.n) / (i + 1))
        
        # Get new state and reward from environment
        new_state, reward, done, _ = env.step(action)
        
        # Update Q-Table with new knowledge using decay rate
        Q[state,action] = reward + dis * np.max(Q[new_state, :])
        
        rAll += reward
        state = new_state
    rList.append(rAll)
    
```